## Remove y_true, y_score and y_binary_score from the saved metrics file

In [2]:
import pandas as pd
import os
import cPickle as pickle
import matplotlib.pylab as plt
from collections import defaultdict
import re
import numpy as np
from thesis.utils.metrics import *

In [3]:
ROOT_RESULTS_LOCATION = '/big/s/shalaby/parameter_search_doc2vec_models_new/sample_0.01/'
# ROOT_RESULTS_LOCATION = '/big/s/shalaby/parameter_search_doc2vec_models_new/full/'
CLASSIFICATION_TYPE = 'sections'
METRICS_PNG_FILE = '{}_validation_metrics.png'
METRICS_PKL_FILE = '{}_training_metrics.pkl'
DOC2VEC_FILES_PREFIX = 'model'
WORD2VEC_METRICS_FILE = 'word2vec_metrics.pkl'
EPOCH_PREFIX = 'epoch_{}'

DOC2VEC_MAX_EPOCHS = 20

CLASSIFICATION_SEPARATOR = ' ** '

CLASSIFICATION_TYPES_TO_USE = ['sections']
METRICS_TO_USE = ['coverage_error', 'f1_micro', 'f1_macro', 'top_3']
METRICS_BEST_VALUE_FUNC = {'coverage_error': np.min, 'f1_micro': np.max, 'f1_macro': np.max, 'top_3': np.max}
METRICS_BEST_INDEX_FUNC = {'coverage_error': np.argmin, 'f1_micro': np.argmax, 'f1_macro': np.argmax, 'top_3': np.argmax}

In [4]:
def get_subdirectories(d):
    #return filter(os.path.isdir, [f for f in os.listdir(d)])
    return [f for f in os.listdir(d) if os.path.isdir(os.path.join(d,f))]
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower() 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(l, key = alphanum_key)

In [5]:
get_subdirectories(ROOT_RESULTS_LOCATION)

['doc2vec_size_300_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'vocab_model_hs',
 'vocab_model',
 'doc2vec_size_200_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_500_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_2000_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_500_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_1000_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_100_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_1_neg_0_vocabsize_None',
 'doc2vec_size_200_w_12_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_300_w_8_type

In [6]:
os.listdir(ROOT_RESULTS_LOCATION)

['doc2vec_size_300_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'vocab_model_hs',
 'vocab_model',
 'doc2vec_size_200_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_500_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_2000_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_500_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_1000_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_100_vocabsize_None',
 'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_1_neg_0_vocabsize_None',
 'doc2vec_size_200_w_12_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None',
 'doc2vec_size_300_w_8_type

In [7]:
table_rows = []
table_index = []
for doc2vec_method in natural_sort(get_subdirectories(ROOT_RESULTS_LOCATION)):
    print doc2vec_method
    epochs_path = os.path.join(ROOT_RESULTS_LOCATION, doc2vec_method)
    # this will have the structure dict of classifiers -> dict of metrics -> list of values throughout the epoch
    epoch_metrics = dict()
    epoch_word2vec_results = []
    epochs_trained = 0
    for epoch in natural_sort(get_subdirectories(epochs_path)):
        print epoch
        epochs_trained += 1
        epoch_path =  os.path.join(epochs_path, epoch)
        if os.path.exists(os.path.join(epoch_path, WORD2VEC_METRICS_FILE)):
            method_word2vec_results = pickle.load(open(os.path.join(epoch_path, WORD2VEC_METRICS_FILE), 'r'))
            epoch_word2vec_results.append(method_word2vec_results)
        for classifier in get_subdirectories(epoch_path):
            # the values of a metric for a specific classifier throughout the epochs
            if not epoch_metrics.get(classifier): epoch_metrics[classifier] = defaultdict(list)
            
            for classf_type in CLASSIFICATION_TYPES_TO_USE:
                classifier_metrics_file = os.path.join(epoch_path, classifier, METRICS_PKL_FILE.format(classf_type))
                if os.path.exists(classifier_metrics_file):
                    classifier_metrics = pickle.load(open(classifier_metrics_file, 'r'))
                    
                    # Fix for adding the y_true, y_score and y_binary_score matrices to the metrics dump which inflated its size
                    if type(classifier_metrics) is list:
                        classifier_metrics = classifier_metrics[-1]
                    if classifier_metrics.get('y_true') is not None:
                        print 'y_true not None'
                        classifier_metrics.pop('y_true', None)
                        classifier_metrics.pop('y_score', None)
                        classifier_metrics.pop('y_binary_score', None)
                        pickle.dump(classifier_metrics, open(classifier_metrics_file, 'w'))
                    
                    # due to a mistake we made, some metrics files contain the whole history of metrics throughout the epochs
                    if type(classifier_metrics) is list:
                        classifier_metrics = classifier_metrics[-1]
                        pickle.dump(classifier_metrics, open(classifier_metrics_file, 'w'))
                    for metric_name in METRICS_TO_USE:
                        metric_full_name = classf_type + CLASSIFICATION_SEPARATOR + metric_name
                        epoch_metrics[classifier][metric_full_name].append(classifier_metrics[metric_name])
                        
    
    total = 0
    word2vec_epoch_correct_results = []
    for res in epoch_word2vec_results:
        total_result = [d for d in res if d['section'] == 'total'][0]
        total_correct = len(total_result['correct'])
        total_incorrect = len(total_result['incorrect'])
        if total == 0: total = total_correct + total_incorrect
        word2vec_epoch_correct_results.append(total_correct)
    best_word2vec_value = np.max(word2vec_epoch_correct_results) if len(word2vec_epoch_correct_results) else np.NaN
    best_word2vec_epoch = np.argmax(word2vec_epoch_correct_results) if len(word2vec_epoch_correct_results) else np.NaN
        
    # now that we are done with our epochs, lets figure out which epoch had the best value of every metric
    for classifier_name in sorted(epoch_metrics.keys()):
        classifier_metrics_dict = defaultdict(float)
        for metric_name in epoch_metrics[classifier_name].keys():
#             best_epoch = np.argmax(epoch_metrics[classifier_name][metric_name])
#             best_value = np.max(epoch_metrics[classifier_name][metric_name])
            just_metric = metric_name.split(CLASSIFICATION_SEPARATOR)[1]
            best_epoch = METRICS_BEST_INDEX_FUNC[just_metric](epoch_metrics[classifier_name][metric_name])
            best_value = METRICS_BEST_VALUE_FUNC[just_metric](epoch_metrics[classifier_name][metric_name])
            metric_range = abs(np.max(epoch_metrics[classifier_name][metric_name]) - np.min(epoch_metrics[classifier_name][metric_name]))
            classifier_metrics_dict[metric_name] = round(best_value, 3)
            classifier_metrics_dict[metric_name + ' -> epoch'] = best_epoch + 1
            classifier_metrics_dict[metric_name + ' range'] = round(metric_range, 3)

        classifier_metrics_dict['word2vec result'] = best_word2vec_value
        classifier_metrics_dict['word2vec result epoch'] = best_word2vec_epoch
        classifier_metrics_dict['Epochs'] = epochs_trained

        table_rows.append(classifier_metrics_dict)
        table_index.append(doc2vec_method + ' / ' + classifier_name)
        
            
df = pd.DataFrame(data=table_rows, index=table_index)            
df

doc2vec_size_100_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None
epoch_1
y_true not None
epoch_2
y_true not None
epoch_3
y_true not None
epoch_4
y_true not None
epoch_5
y_true not None
epoch_6
y_true not None
epoch_7
y_true not None
epoch_8
y_true not None
epoch_9
y_true not None
epoch_10
y_true not None
epoch_11
y_true not None
epoch_12
y_true not None
epoch_13
y_true not None
epoch_14
y_true not None
epoch_15
y_true not None
epoch_16
y_true not None
epoch_17
y_true not None
epoch_18
y_true not None
epoch_19
y_true not None
epoch_20
y_true not None
doc2vec_size_200_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None
epoch_1
y_true not None
epoch_2
y_true not None
epoch_3
y_true not None
epoch_4
y_true not None
epoch_5
y_true not None
epoch_6
y_true not None
epoch_7
y_true not None
epoch_8
y_true not None
epoch_9
y_true not None
epoch_10
y_true not None
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_Non

,Epochs,sections ** coverage_error,sections ** coverage_error -> epoch,sections ** coverage_error range,sections ** f1_macro,sections ** f1_macro -> epoch,sections ** f1_macro range,sections ** f1_micro,sections ** f1_micro -> epoch,sections ** f1_micro range,sections ** top_3,sections ** top_3 -> epoch,sections ** top_3 range,word2vec result,word2vec result epoch
doc2vec_size_100_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.01_classweights_balanced,20,3.310,20,0.185,0.535,20,0.020,0.619,20,0.021,0.865,20,0.024,0,0
doc2vec_size_200_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.1_classweights_balanced,10,3.472,10,0.071,0.513,10,0.008,0.592,10,0.009,0.833,9,0.013,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.001_classweights_None,20,4.336,20,0.500,0.463,20,0.078,0.656,20,0.054,0.784,20,0.046,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.001_classweights_balanced,20,3.220,20,0.162,0.549,20,0.017,0.635,20,0.020,0.880,20,0.017,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.01_classweights_balanced,20,3.250,20,0.157,0.543,20,0.018,0.628,20,0.019,0.873,20,0.021,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.1_classweights_balanced,20,3.409,20,0.147,0.519,20,0.015,0.597,20,0.016,0.839,20,0.022,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_100_vocabsize_None / svm_iter_100_reg_0.001_classweights_balanced,15,3.391,15,0.116,0.530,15,0.010,0.614,15,0.012,0.860,15,0.011,0,0
doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_1_neg_0_vocabsize_None / svm_iter_100_reg_0.001_classweights_balanced,5,3.494,5,0.046,0.519,5,0.004,0.602,5,0.005,0.849,5,0.005,0,0
doc2vec_size_200_w_12_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_100_reg_0.01_classweights_balanced,13,3.324,12,0.080,0.535,12,0.008,0.619,12,0.009,0.864,12,0.010,0,0
doc2vec_size_300_w_5_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None / svm_iter_1000_reg_0.001_classweights_balanced,20,3.186,20,0.152,0.554,20,0.017,0.640,20,0.018,0.884,20,0.017,0,0
